In [42]:
import os
import json

In [43]:
# load an example text with duplicated content
path = "/srv/data/corpus-corporum/cc_sents_jsons_duplicated/"
id = 10787
fn = str(id) + ".json"

In [44]:
sents_data = json.load(open(path + fn, "rb"))

In [46]:
sents_data[:2]

[['10787',
  0,
  'Epistola ad Gaufridum Richardum et monachos Sauignienses Reuerendissimo domino et Dei seruo abbati Sauigniensi Gaufrido, nec non et ejusdem coenobii uenerando priori Richardo, uniuersoque conuentui, Joannes Constantiensis sic de uirtute in uirtutem proficere, ut Deum deorum mereantur in Sion uidere.',
  [['Epistola', 'epistola', 'NOUN', [0, 8]],
   ['ad', 'ad', 'ADP', [9, 11]],
   ['Gaufridum', 'Gaufridus', 'PROPN', [12, 21]],
   ['Richardum', 'Richardus', 'PROPN', [22, 31]],
   ['et', 'et', 'CCONJ', [32, 34]],
   ['monachos', 'monachus', 'NOUN', [35, 43]],
   ['Sauignienses', 'Sauignienses', 'PROPN', [44, 56]],
   ['Reuerendissimo', 'Reuerendus', 'ADJ', [57, 71]],
   ['domino', 'dominus', 'NOUN', [72, 78]],
   ['et', 'et', 'CCONJ', [79, 81]],
   ['Dei', 'Deus', 'PROPN', [82, 85]],
   ['seruo', 'seruus', 'NOUN', [86, 91]],
   ['abbati', 'abbas', 'NOUN', [92, 98]],
   ['Sauigniensi', 'Sauigniensus', 'PROPN', [99, 110]],
   ['Gaufrido', 'Gaufridum', 'PROPN', [111, 119]

In [52]:
path = "/srv/data/corpus-corporum/cc_sents_jsons_duplicated/"
id = 10012
fn = str(id) + ".json"
sents_data = json.load(open(path + fn, "rb"))
len(sents_data)

13992

In [53]:
sents_data[:3]

[['10012',
  0,
  'Admonitio Praeuia.',
  [['Admonitio', 'admonitio', 'NOUN', [0, 9]],
   ['Praeuia', 'praeuius', 'PROPN', [10, 17]],
   ['.', '.', 'PUNCT', [17, 18]]]],
 ['10012',
  1,
  'Postquam simoniam cum ipso parente suo Simone Mago spirituali jaculo confodit princeps apostolorum, tam horrendi monstri ne nomen quidem apud Christianos audiri deberet.',
  [['Postquam', 'postquam', 'SCONJ', [0, 8]],
   ['simoniam', 'simonia', 'NOUN', [9, 17]],
   ['cum', 'cum', 'ADP', [18, 21]],
   ['ipso', 'ipse', 'DET', [22, 26]],
   ['parente', 'parens', 'NOUN', [27, 34]],
   ['suo', 'suus', 'DET', [35, 38]],
   ['Simone', 'Simo', 'PROPN', [39, 45]],
   ['Mago', 'Mago', 'PROPN', [46, 50]],
   ['spirituali', 'spiritualis', 'ADJ', [51, 61]],
   ['jaculo', 'iaculus', 'NOUN', [62, 68]],
   ['confodit', 'confodio', 'VERB', [69, 77]],
   ['princeps', 'princeps', 'NOUN', [78, 86]],
   ['apostolorum', 'apostolus', 'NOUN', [87, 98]],
   [',', ',', 'PUNCT', [98, 99]],
   ['tam', 'tam', 'ADV', [100, 103]],

In [51]:
def deduplicate_sents_data(sents_data):
    block_size = 3
    i = 0
    while i <= len(sents_data) - block_size:
        current_block = [s[2] for s in sents_data[i:i + block_size]]

        # Search for repeating block
        found_duplicate = False
        for j in range(i + 1, len(sents_data) - block_size + 1):
            comparison_block = [s[2] for s in sents_data[j:j + block_size]]
            if current_block == comparison_block:
                # Delete the second block
                del sents_data[j:j + block_size]
                found_duplicate = True
                break  # Stop scanning once one duplicate is removed

        if not found_duplicate:
            i += 1  # Advance if no match was found
    return sents_data

In [54]:
len(sents_data)

13992

In [55]:
deduplicated_sents_data = deduplicate_sents_data(sents_data)
len(deduplicated_sents_data)

4755

In [56]:
# make a new dir, but skip if it already exists
target_path = "/srv/data/corpus-corporum/cc_sents_jsons/"
os.makedirs(target_path, exist_ok=True)

In [57]:
%%time
source_path = "/srv/data/corpus-corporum/cc_sents_jsons_duplicated/"
fns = os.listdir(source_path)
for fn in fns:
    sents_data = json.load(open(path + fn, "rb"))
    sents_data = deduplicate_sents_data(sents_data)
    with open(target_path + fn, "w") as f:
        json.dump(sents_data, f)


CPU times: user 1d 7h 36min 34s, sys: 57.3 s, total: 1d 7h 37min 31s
Wall time: 1d 7h 38min 10s


In [ ]:
source_jsons = "/srv/data/corpus-corporum/cc_sents_jsons/"
target_jsons = "/srv/data/corpus-corporum/sents_jsons_dicts/"
os.makedirs(target_jsons, exist_ok=True)

for fn in os.listdir(source_jsons):
    doc_id = fn.rpartition(".")[0]
    with open(source_jsons + fn, "r") as f:
        sents_data = json.load(f)  # list of (sent_text, tokens)

    sents_data_dicts = []
    for doc_id, sent_n, sent_text, tokens in sents_data:
        tokens_data_dicts = []
        for tok in tokens:
            # tok is (token_text, lemma, pos, (char_start, char_end))
            token_text, lemma, pos, span = tok
            char_start, char_end = int(span[0]), int(span[1])

            tokens_data_dicts.append({
                "token_text": token_text,
                "lemma": lemma,
                "pos": pos,
                "ref": {},                 # always empty dict as requested
                "char_start": char_start,
                "char_end": char_end,
            })

        sents_data_dicts.append({
            "work_id": doc_id,            # from filename
            "sent_id": sent_n,            # enumerated
            "sent_text": sent_text,
            "tokens_data": tokens_data_dicts,
        })

    with open(os.path.join(target_jsons,  fn), "w", encoding="utf-8") as f:
        json.dump(sents_data_dicts, f, ensure_ascii=False, indent=2)